In [2]:
### Загружаем библиотеки и данные
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from scipy.stats import norm
 
data = load_iris()
X, y, column_names = data['data'], data['target'], data['feature_names']
X = pd.DataFrame(X, columns = column_names)

In [3]:
### Разбиваем данные
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=44)
 
means = X_train.groupby(y_train).apply(np.mean)
stds = X_train.groupby(y_train).apply(np.std)
 
### Вычисляем априорную вероятность класса
probs = X_train.groupby(y_train).apply(lambda x: len(x)) / X_train.shape[0]
 

/Users/igorpostoev/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [ ]:
### Вычисляем вероятность для Теоремы Байеса для каждого элемента
y_pred = []
# каждый элемент в валидационной части данных
for elem in range(X_val.shape[0]):
   p = {}
 
   # для каждого возможного класса
   for cl in np.unique(y_train):
 
       # априорная вероятность взятого ранее класса
       p[cl] = probs.iloc[cl]
 
       # для каждого столбца в датасете
       for index, param in enumerate(X_val.iloc[elem]):
 
           # умножаем вероятность того, что данное значение столбца
           # будет принадлежать распределению для выбранного класса
           p[cl] *= norm.pdf(param, means.iloc[cl, index], stds.iloc[cl, index])
  
   y_pred.append(pd.Series(p).values.argmax())
 
### Посмотрим точность нашего предсказания несколькими методами
# ручной классификатор
from sklearn.metrics import accuracy_score
accuracy1 = accuracy_score(y_val, y_pred)
 
# классификатор из библиотеки sklearn
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train)
accuracy2 = accuracy_score(y_val, model.predict(X_val))

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from scipy.stats import norm

In [ ]:
 # для начала определяется малый набор данных с произвольно взятой длиной словаря мешка слов

In [273]:
X_train = pd.DataFrame(np.array([[0, 1, 0],
                                   [0, 0, 1],
                                   [0, 0, 1],
                                   [1, 1, 0],
                                   [1, 0, 1],
                                    [0, 0, 0]]))
y_train = np.array([0, 1, 1, 0, 1, 0])

X_val = pd.DataFrame(np.array([[0, 1, 0],
                               [0, 0, 1]]))
y_val = np.array([0, 1])

#P(Y)
aprior_class_probs = X_train.groupby(y_train).apply((lambda x: len(x))) / len(X_train)

#P(X|Y) / P(X)
aprior_feature_probs = {}
for feature in X_train.columns:
    aprior_feature_probs[feature] = np.count_nonzero(X_train[feature].values == 1)

groups = X_train.groupby(y_train).groups
w_prob_by_cl = {}
for cl in np.unique(y_train):
    w_prob_by_cl[cl] = {}
    for feature in X_val.columns:
        cond_feature_p = np.count_nonzero(X_train.iloc[groups[cl]][feature].values == 1)
        cond_feature_p = cond_feature_p / aprior_feature_probs[feature]
        w_prob_by_cl[cl][feature] = cond_feature_p
    
y_pred = []
for entry in range(len(X_val.values)):
    p = {}
    
    # определение условной вероятности каждого класса при наличии признаков P(Y|X)
    for cl in np.unique(y_val):
        # априорная вероятность класса - отношения вхождений соответствующих данному классу ко всем вхождениям P(Y)
        p[cl] = aprior_class_probs[cl]
        
        # апостериорная вероятность всех признаков при данном классе равна
        # произведению(сейчас сумма, т к если одного из слов не будет в предложении, вероятность всего класса будет 0)
        # условных вероятностей для каждого признака по отдельности (потому алгоритм и называют "наивным")
        
        # условная вероятность в данном случае - отношение количества вхождений, содержащих данный признак к общему количеству вхождений(для данного класса)
        for feature in X_val.columns:
            if X_val.iloc[entry][feature] == 1:
                p[cl] += w_prob_by_cl[cl][feature]
                print(f"{cl}:{feature} - {p[cl]}")
        
    y_pred.append(pd.Series(p).values.argmax())  